# HW2 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  2
* **Date:**  5/23/16

**HW2.1.** Sort in Hadoop MapReduce.  **Given as input:** Records of the form (integer, “NA”), where integer is any integer, and “NA” is just the empty string.  **Output:** sorted key value pairs of the form (integer, “NA”) in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.

**Part 1** Write code to generate N  random records of the form (integer, “NA”). Let N = 10,000.

**Part 2** Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers

In [2]:
import random

N=10000
min_int = -100000
max_int = 100000
filename = "random_integers.txt"
with open(filename, 'w') as f:
    for i in range(N):
        f.write('<%d, "">\n' % (random.randint(min_int, max_int)))

In [9]:
# Create a directory in HDFS to store the data for this problem
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user/hadoop

16/05/20 22:10:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# Read the input file into HDFS
#!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal random_integers.txt /user/hadoop

16/05/20 22:11:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - hadoop supergroup          0 2016-05-20 22:10 /user/hadoop/outputHW2-1
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [22]:
# Check to make sure the file is there.
#!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/20 22:33:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [17]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.1.

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove the angle brackets
    line = line.strip('<>\n')
    # split the line into words
    words = line.split(',')
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s' % (words[0]))

Overwriting mapper.py


In [43]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.1.

import sys

N=10000
i=0
# input comes from STDIN
for line in sys.stdin:
    if (i>=0 and i<10) or (i>=(N-10) and i<N):
        print('<%s, "">' % (line.strip()))
    i += 1

Overwriting reducer.py


In [44]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options=-nr \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:00:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/outputHW2-1/*': No such file or directory
16/05/21 14:00:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:00:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar4921037629859414604/] [] /tmp/streamjob3968137248463036228.jar tmpDir=null
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/05/21 14:00:51 INFO mapreduce.JobSubmitter: number of splits:2
16/05/21 14:00:51 INFO Configuration.deprecati

In [45]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:01:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:01 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup        270 2016-05-21 14:01 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:01:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<99999, "">	
<99995, "">	
<99982, "">	
<99974, "">	
<99969, "">	
<99942, "">	
<99929, "">	
<99892, "">	
<99887, "">	
<99882, "">	
<-99775, "">	
<-99842, "">	
<-99867, "">	
<-99889, "">	
<-99912, "">	
<-99939, "">	
<-99943, "">	
<-99961, "">	
<-99984, "">	
<-99994, "">	


In [46]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.2.

# Import sys library to access arguments passed in when the module is called
import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip('<>\n')
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [47]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.2

from operator import itemgetter
# Import sys library to access arguments passed in when the module is called
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [48]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:06:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/_SUCCESS
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar7496711587575795057/] [] /tmp/streamjob8403600677380589246.jar tmpDir=null
16/05/21 14:06:23 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205

In [49]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:06:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup      91627 2016-05-21 14:06 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
""	10000
-10013,	1
-10041,	1
-10059,	1
-10068,	2
-10090,	1
-10095,	1
-10116,	1
-10132,	1
-10164,	1
-10181,	1
-10227,	1
-10235,	1
-10246,	1
-10247,	1
-1026,	1
-10260,	1
-10273,	1
-10287,	1
-10306,	2
-10307,	1
-10333,	1
-10341,	1
-10348,	1
-10392,	1
-10405,	1
-10414,	1
-10421,	1
-10436,	1
-10454,	1
-1046,	1
-10465,	1
-10493,	1
-10497,	1
-10518,	1
-10545,	1
-10568,	1
-10585,	1
-10611,	1
-10660,	1
-10671,	1
-10686,	1
-10691,	1
-10710,	1
-10720,	1
-10766,	1
-10774,	1
-10781,	1
-10806,	1
-10822,	1
-

**HW2.3.**
Training Error: 0.000000	
count,zeroprob	5617	4960


In [51]:
# Read the input file into HDFS
#!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal enronemail_1h.txt /user/hadoop
# Check to make sure the file is there
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/21 14:24:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   3 hadoop supergroup     203981 2016-05-21 14:24 /user/hadoop/enronemail_1h.txt
drwxr-xr-x   - hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [1]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 1 code for HW2.3

# Import print function from python 3
from __future__ import print_function
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
# Variable vocab:  stores the unique terms used in this file
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Update 'count_docs' by 1 for the class of this document
    count_docs[TRUTH] += 1
    # Update 'count_words' by the length of the 'words_all' list for the class of this document
    count_words[TRUTH] += len(words_all)
    # Create a variable to store the unique words from this line.
    vocab_line = []
    # Loop through each of the terms in the words of this line
    for term in words_all:
        # Update the vocab for this file
        if term not in vocab:
            vocab.append(term)
        # Update the vocab for this line
        if term not in vocab_line:
            vocab_line.append(term)
        # Initialize a list to store the count of this term
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [0,0]
    # For each of the words in the vocabulary print out the 'term' and the '# of occurances
    # of the term' for reducer.py
    for term in vocab_line:
        # Store the number of occurances of the vocab word in the document
        count_term = words_all.count(term)
        # Update 'count_vocab_terms' for the class of this document
        count_vocab_terms[term][TRUTH] += count_term

# Print the count totals for each class that get passed to reducer.py
# Print the document count from this part of the file for Ham and Spam
print('count,docs\t%d\t%d' % (count_docs[0], count_docs[1]))
# Print the word count from this part of the file for Ham and Spam
print('count,words\t%d\t%d' % (count_words[0], count_words[1]))
# Print each term in the vocab and then its count for Ham and Spam
for term in vocab:
    print('%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: Reducer 1 code for HW2.3

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line by tabs
    elements = line.split("\t")
    # The 1st value in the line is the ID of the document
    label = elements[0]
    # Update the 'count_docs' variable with the numbers from the file for each class
    if label == 'count,docs':
        count_docs[0] += int(elements[1])
        count_docs[1] += int(elements[2])
    # Update the 'count_words' variable with the numbers from the file for each class
    elif label == 'count,words':
        count_words[0] += int(elements[1])
        count_words[1] += int(elements[2])
    # Update the 'count_vocab_terms' variable with the numbers from the file for each class
    else:
        term = label
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [int(elements[1]), int(elements[2])]
        else:
            count_vocab_terms[term][0] += int(elements[1])
            count_vocab_terms[term][1] += int(elements[2])

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Use smoothing by adding 1.0 to the numerator and the denominator in the condprob equation.
# Create a variable to control if smoothing is used.
SMOOTHING = False
# Print the prior probabilities for each class
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
print('prior,prob\t%f\t%f' % (prior[0], prior[1]))

# Print the conditional probabilities for each term for Ham and Spam
for term in count_vocab_terms:
    for cls in classes:
        # If using smoothing, then add 1.0 to the numerator and the denominator in the condprob equation.
        if SMOOTHING:
            condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)
        # If NOT using smoothing, then leave the numerator and the denominator alone.
        else:
            condprob[term][cls] = (count_vocab_terms[term][cls]) / (float(count_words[cls]))
    print('%s\t%f\t%f' % (term, condprob[term][0], condprob[term][1]))

Overwriting reducer.py


In [3]:
# Remove output from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-3a/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-3a
# Run a hadoop streaming job to create the model
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-3a

16/05/24 16:10:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:10:56 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3a/_SUCCESS
16/05/24 16:10:56 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3a/part-00000
16/05/24 16:10:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:10:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar6812880909108617658/] [] /tmp/streamjob3150896271925999271.jar tmpDir=null
16/05/24 16:10:59 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.2

In [4]:
# Reivew the output files obtained from the above job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-3a
# Look at the contents of the output of the reducer in the part-00000 file in HDFS
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-3a/part-00000

16/05/24 16:11:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 16:11 /user/hadoop/outputHW2-3a/_SUCCESS
-rw-r--r--   3 hadoop supergroup     140783 2016-05-24 16:11 /user/hadoop/outputHW2-3a/part-00000
16/05/24 16:11:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
prior,prob	0.560000	0.440000
yellow	0.000000	0.000054
barraged	0.000000	0.000054
prefix	0.000000	0.000054
railing	0.000000	0.000054
increase	0.000422	0.000326
granting	0.000000	0.000054
electricity	0.000070	0.000000
originality	0.000000	0.000054
homemakers	0.000000	0.000054
hormone	0.000000	0.000109
regional	0.000774	0.000000
screaming	0.000000	0.000054
four	0.000211	0.000272
internally	0.000141	0.000000
prize	0.000000	0.000054
customizable	0.000000	0.000054
wednesday	0.000422	0.000000
cyber

In [3]:
# Copy the model output from the 1st reducer to the local file system
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-3a/part-00000 model.txt

16/05/21 23:29:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 23:29:10 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [9]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 2 code for HW2.3

# Import print function from python 3
from __future__ import print_function
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Initialize the 'condprob' variable to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}

# Load the model from the model.txt file.
# 1.  prior:  stores the prior probabilites for Ham (0) and Spam (0)
# 2.  condprob:  stores the conditional probabilities for each term for Ham (0) and Spam (0)
with open("model.txt", 'r') as f:
    for line in f.readlines():
        elements = line.split('\t')
        if elements[0] == 'prior,prob':
            prior = [float(elements[1]), float(elements[2])]
        else:
            term = elements[0]
            condprob[term] = [float(elements[1]), float(elements[2])]

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]

for line in sys.stdin:
    # Read in the lines from the input file
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Make a prediction for each document based on the model
    score = [0, 0]
    for cls in classes:
        # Initialize the score with the prior probability for the class
        score[cls] = math.log(prior[cls])
        for term in words_all:
            # Add the conditional probability to the scoare for each term.
            # If any of the conditional probabilities for the document are 0, then set the score
            # to -100000000 because, in actuality the probabilities are being multiplied, so if one is 0
            # then the whole score will be 0.  If this happens, set the score to a very low number so that
            # this class will have a very low score and won't be selected and break out of the loop
            if condprob[term][cls] == 0.0:
                score[cls] = -1000000000
                count_docs_zero_prob[cls] += 1
            else:
                score[cls] += math.log(condprob[term][cls])
    # Set the prediction for the document to the class that has the highest score
    prediction = 1 if (score[1] > score[0]) else 0
    # Print the document ID, the real classification, the predicted classification and the score
    # for each class for the reducer
    print("%s\t%d\t%d\t%f\t%f" % (ID, TRUTH, prediction, score[0], score[1]))

# Print the number of times a zero probability for reporting purposes.
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))

Overwriting mapper.py


In [6]:
%%writefile reducer.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Reducer 2 code for HW2.3

import sys

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]
# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Initialize a dictionary to store the IDs and the scores from the predictions
doc_probs = {}

# Input comes from STDIN
for line in sys.stdin:
    # Split the line by tabs
    items = line.split('\t')
    # Store the zero probability counts
    if items[0] == 'count,zeroprob':
        count_docs_zero_prob[0] += int(items[1])
        count_docs_zero_prob[1] += int(items[2])
    # Update the incorrect and total counts based on the accuracy of the prediction
    # Store the scores for each class in the doc_probs dictionary for reporting purposes
    else:
        ID = items[0]
        TRUTH = int(items[1])
        prediction = int(items[2])
        if TRUTH != prediction:
            count_incorrect += 1
        count_total += 1
        doc_probs[ID] = [float(items[3]), float(items[4])]

# Print the Training Error, zero probability counts and scores for reporting purposes
print('Training Error: %f' % (count_incorrect / float(count_total)))
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))
for ID, probs in doc_probs.iteritems():
    print('%s\t%f\t%f' % (ID, probs[0], probs[1]))

Overwriting reducer.py


In [10]:
# Remove the outputs from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-3b/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-3b
# Call a hadoop streaming job to classify the documents.  Pass in the model.txt file so it is available
# to all of the mappers
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py,model.txt -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-3b

16/05/24 16:14:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:14:22 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3b/_SUCCESS
16/05/24 16:14:22 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3b/part-00000
16/05/24 16:14:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:14:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar6040676337692468171/] [] /tmp/streamjob6443169919537839235.jar tmpDir=null
16/05/24 16:14:25 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.2

In [11]:
# Inspect the output produced from the mapreduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-3b
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-3b/part-00000

16/05/24 16:14:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 16:14 /user/hadoop/outputHW2-3b/_SUCCESS
-rw-r--r--   3 hadoop supergroup       5474 2016-05-24 16:14 /user/hadoop/outputHW2-3b/part-00000
16/05/24 16:14:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Training Error: 0.000000	
count,zeroprob	5617	4960
0010.2003-12-18.GP	-1000000008.866751	-55.316830
0010.2001-06-28.SA_and_HP	-1000000000.000000	-3456.055866
0001.2000-01-17.beck	-3628.592339	-1000000000.000000
0018.1999-12-14.kaminski	-988.159991	-1000000000.000000
0005.1999-12-12.kaminski	-777.593930	-1000000000.000000
0011.2001-06-29.SA_and_HP	-1000000005.372825	-15962.696793
0008.2004-08-01.BG	-1000000000.000000	-5938.923501
0009.1999-12-14.farmer	-589.805873	-1000000000.000000
0017.2003

In [43]:
# Copy the model output from the 2nd reducer to the local file system for reporting purposes.
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-3b/part-00000 doc_scores.txt

16/05/22 00:52:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/22 00:52:06 WARN hdfs.DFSClient: DFSInputStream has been closed already


**HW2.4** Repeat HW2.3 with the following modification: use Laplace plus-one smoothing. Compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.

ANSWER:  
Training Error for HW2.3: 0% (0/100 documents misclassified)  
Training Error for HW2.4: 0% (0/100 documents misclassified)

Using Laplace plus-one smoothing causes no difference in the training error rate.  Smoothing does not change any results in this case because there are no term-class combinations that did not occur in the training data.  Smoothing only makes a difference when there is a new term-class combination that would have been 0 in the training data had we not added 1 to it.

In [29]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 1 code for HW2.4

# Import print function from python 3
from __future__ import print_function
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
# Variable vocab:  stores the unique terms used in this file
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Update 'count_docs' by 1 for the class of this document
    count_docs[TRUTH] += 1
    # Update 'count_words' by the length of the 'words_all' list for the class of this document
    count_words[TRUTH] += len(words_all)
    # Create a variable to store the unique words from this line.
    vocab_line = []
    # Loop through each of the terms in the words of this line
    for term in words_all:
        # Update the vocab for this file
        if term not in vocab:
            vocab.append(term)
        # Update the vocab for this line
        if term not in vocab_line:
            vocab_line.append(term)
        # Initialize a list to store the count of this term
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [0,0]
    # For each of the words in the vocabulary print out the 'term' and the '# of occurances
    # of the term' for reducer.py
    for term in vocab_line:
        # Store the number of occurances of the vocab word in the document
        count_term = words_all.count(term)
        # Update 'count_vocab_terms' for the class of this document
        count_vocab_terms[term][TRUTH] += count_term

# Print the count totals for each class that get passed to reducer.py
# Print the document count from this part of the file for Ham and Spam
print('count,docs\t%d\t%d' % (count_docs[0], count_docs[1]))
# Print the word count from this part of the file for Ham and Spam
print('count,words\t%d\t%d' % (count_words[0], count_words[1]))
# Print each term in the vocab and then its count for Ham and Spam
for term in vocab:
    print('%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [30]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: Reducer 1 code for HW2.4

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line by tabs
    elements = line.split("\t")
    # The 1st value in the line is the ID of the document
    label = elements[0]
    # Update the 'count_docs' variable with the numbers from the file for each class
    if label == 'count,docs':
        count_docs[0] += int(elements[1])
        count_docs[1] += int(elements[2])
    # Update the 'count_words' variable with the numbers from the file for each class
    elif label == 'count,words':
        count_words[0] += int(elements[1])
        count_words[1] += int(elements[2])
    # Update the 'count_vocab_terms' variable with the numbers from the file for each class
    else:
        term = label
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [int(elements[1]), int(elements[2])]
        else:
            count_vocab_terms[term][0] += int(elements[1])
            count_vocab_terms[term][1] += int(elements[2])

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Use smoothing by adding 1.0 to the numerator and the denominator in the condprob equation.
# Create a variable to control if smoothing is used.
# For 2.4 set SMOOTHING to True
SMOOTHING = True
# Print the prior probabilities for each class
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
print('prior,prob\t%f\t%f' % (prior[0], prior[1]))

# Print the conditional probabilities for each term for Ham and Spam
for term in count_vocab_terms:
    for cls in classes:
        # If using smoothing, then add 1.0 to the numerator and the denominator in the condprob equation.
        if SMOOTHING:
            condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)
        # If NOT using smoothing, then leave the numerator and the denominator alone.
        else:
            condprob[term][cls] = (count_vocab_terms[term][cls]) / (float(count_words[cls]))
    print('%s\t%f\t%f' % (term, condprob[term][0], condprob[term][1]))

Overwriting reducer.py


In [31]:
# Remove output from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-4a/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-4a
# Run a hadoop streaming job to create the model
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-4a

16/05/24 16:40:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:40:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-4a/_SUCCESS
16/05/24 16:40:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-4a/part-00000
16/05/24 16:40:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:40:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar4857434765470578216/] [] /tmp/streamjob760824675251139049.jar tmpDir=null
16/05/24 16:40:09 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.20

In [32]:
# Reivew the output files obtained from the above job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-4a
# Look at the contents of the output of the reducer in the part-00000 file in HDFS
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-4a/part-00000

16/05/24 16:40:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 16:40 /user/hadoop/outputHW2-4a/_SUCCESS
-rw-r--r--   3 hadoop supergroup     140783 2016-05-24 16:40 /user/hadoop/outputHW2-4a/part-00000
16/05/24 16:40:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
prior,prob	0.560000	0.440000
yellow	0.000070	0.000109
barraged	0.000070	0.000109
prefix	0.000070	0.000109
railing	0.000070	0.000109
increase	0.000492	0.000381
granting	0.000070	0.000109
electricity	0.000141	0.000054
originality	0.000070	0.000109
homemakers	0.000070	0.000109
hormone	0.000070	0.000163
regional	0.000844	0.000054
screaming	0.000070	0.000109
four	0.000281	0.000326
internally	0.000211	0.000054
prize	0.000070	0.000109
customizable	0.000070	0.000109
wednesday	0.000492	0.000054
cyber

In [33]:
# Copy the model output from the 1st reducer to the local file system
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-4a/part-00000 model24.txt

16/05/24 16:40:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:40:43 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [34]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 2 code for HW2.4

# Import print function from python 3
from __future__ import print_function
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Initialize the 'condprob' variable to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}

# Load the model from the model.txt file.
# 1.  prior:  stores the prior probabilites for Ham (0) and Spam (0)
# 2.  condprob:  stores the conditional probabilities for each term for Ham (0) and Spam (0)
with open("model24.txt", 'r') as f:
    for line in f.readlines():
        elements = line.split('\t')
        if elements[0] == 'prior,prob':
            prior = [float(elements[1]), float(elements[2])]
        else:
            term = elements[0]
            condprob[term] = [float(elements[1]), float(elements[2])]

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]

for line in sys.stdin:
    # Read in the lines from the input file
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Make a prediction for each document based on the model
    score = [0, 0]
    for cls in classes:
        # Initialize the score with the prior probability for the class
        score[cls] = math.log(prior[cls])
        for term in words_all:
            # Add the conditional probability to the scoare for each term.
            # If any of the conditional probabilities for the document are 0, then set the score
            # to -100000000 because, in actuality the probabilities are being multiplied, so if one is 0
            # then the whole score will be 0.  If this happens, set the score to a very low number so that
            # this class will have a very low score and won't be selected and break out of the loop
            if condprob[term][cls] == 0.0:
                score[cls] = -1000000000
                count_docs_zero_prob[cls] += 1
            else:
                score[cls] += math.log(condprob[term][cls])
    # Set the prediction for the document to the class that has the highest score
    prediction = 1 if (score[1] > score[0]) else 0
    # Print the document ID, the real classification, the predicted classification and the score
    # for each class for the reducer
    print("%s\t%d\t%d\t%f\t%f" % (ID, TRUTH, prediction, score[0], score[1]))

# Print the number of times a zero probability for reporting purposes.
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))

Overwriting mapper.py


In [35]:
%%writefile reducer.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Reducer 2 code for HW2.4

import sys

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]
# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Initialize a dictionary to store the IDs and the scores from the predictions
doc_probs = {}

# Input comes from STDIN
for line in sys.stdin:
    # Split the line by tabs
    items = line.split('\t')
    # Store the zero probability counts
    if items[0] == 'count,zeroprob':
        count_docs_zero_prob[0] += int(items[1])
        count_docs_zero_prob[1] += int(items[2])
    # Update the incorrect and total counts based on the accuracy of the prediction
    # Store the scores for each class in the doc_probs dictionary for reporting purposes
    else:
        ID = items[0]
        TRUTH = int(items[1])
        prediction = int(items[2])
        if TRUTH != prediction:
            count_incorrect += 1
        count_total += 1
        doc_probs[ID] = [float(items[3]), float(items[4])]

# Print the Training Error, zero probability counts and scores for reporting purposes
print('Training Error: %f' % (count_incorrect / float(count_total)))
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))
for ID, probs in doc_probs.iteritems():
    print('%s\t%f\t%f' % (ID, probs[0], probs[1]))

Overwriting reducer.py


In [36]:
# Remove the outputs from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-4b/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-4b
# Call a hadoop streaming job to classify the documents.  Pass in the model.txt file so it is available
# to all of the mappers
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py,model24.txt -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-4b

16/05/24 16:41:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:41:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-4b/_SUCCESS
16/05/24 16:41:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-4b/part-00000
16/05/24 16:41:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:41:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar8469822023791319689/] [] /tmp/streamjob3704442890562259351.jar tmpDir=null
16/05/24 16:41:22 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.2

In [37]:
# Inspect the output produced from the mapreduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-4b
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-4b/part-00000

16/05/24 16:41:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 16:41 /user/hadoop/outputHW2-4b/_SUCCESS
-rw-r--r--   3 hadoop supergroup       4820 2016-05-24 16:41 /user/hadoop/outputHW2-4b/part-00000
16/05/24 16:41:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Training Error: 0.000000	
count,zeroprob	0	0
0010.2003-12-18.GP	-53.695743	-52.688604
0010.2001-06-28.SA_and_HP	-3689.417177	-3381.099048
0001.2000-01-17.beck	-3535.918583	-4160.185188
0018.1999-12-14.kaminski	-965.647201	-990.262795
0005.1999-12-12.kaminski	-747.471559	-865.846157
0011.2001-06-29.SA_and_HP	-17518.334352	-15660.798148
0008.2004-08-01.BG	-6063.024332	-5737.480780
0009.1999-12-14.farmer	-571.861332	-666.858911
0017.2003-12-18.GP	-218.324629	-205.870590
0011.2001-06-28.SA_and_H

In [38]:
# Copy the model output from the 2nd reducer to the local file system for reporting purposes.
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-4b/part-00000 doc_scores24.txt

16/05/24 16:44:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 16:44:24 WARN hdfs.DFSClient: DFSInputStream has been closed already


**HW2.5.** Repeat HW2.4. This time when modeling and classification ignore tokens with a frequency of less than three (3) in the training set. How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifier on the training dataset. Report the error and the change in error. HINT: ignore tokens with a frequency of less than three (3). Think of this as a preprocessing step. How many mapreduce jobs do you need to solve this homework?

ANSWER:  After removing tokens with a frequency of less than 3 (i.e. terms whose (ham_count + spam_count) < 3), the model went from having a vocabulary of about 5,490 terms to about 1,862 terms.  After the terms were removed from the vocabulary the total word_counts for each class was decreased accordingly.  After removing the terms, the new training error is 2% (2/100 documents were misclassified).  The training error without removing the terms was 0% (0/100 documents were misclassified).  This is a change of 2%.  This indicates that some of the terms that were removed were important to classifying the documents correctly.

This problem required 2 mapreduce jobs just like problem 2.4.  No additional mapreduce jobs are required.	The extra pass over the terms to remove the terms with freqency less than 3 can be done in the 1st reducer where the model is being built.  This pass loops through the current vocabulary and counts and removes any terms that have a freq less than 3 and then updates the total word counts accordingly.

In [12]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 1 code for HW2.5

# Import print function from python 3
from __future__ import print_function
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
# Variable vocab:  stores the unique terms used in this file
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Update 'count_docs' by 1 for the class of this document
    count_docs[TRUTH] += 1
    # Update 'count_words' by the length of the 'words_all' list for the class of this document
    count_words[TRUTH] += len(words_all)
    # Create a variable to store the unique words from this line.
    vocab_line = []
    # Loop through each of the terms in the words of this line
    for term in words_all:
        # Update the vocab for this file
        if term not in vocab:
            vocab.append(term)
        # Update the vocab for this line
        if term not in vocab_line:
            vocab_line.append(term)
        # Initialize a list to store the count of this term
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [0,0]
    # For each of the words in the vocabulary print out the 'term' and the '# of occurances
    # of the term' for reducer.py
    for term in vocab_line:
        # Store the number of occurances of the vocab word in the document
        count_term = words_all.count(term)
        # Update 'count_vocab_terms' for the class of this document
        count_vocab_terms[term][TRUTH] += count_term

# Print the count totals for each class that get passed to reducer.py
# Print the document count from this part of the file for Ham and Spam
print('count,docs\t%d\t%d' % (count_docs[0], count_docs[1]))
# Print the word count from this part of the file for Ham and Spam
print('count,words\t%d\t%d' % (count_words[0], count_words[1]))
# Print each term in the vocab and then its count for Ham and Spam
for term in vocab:
    print('%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [13]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: Reducer 1 code for HW2.5

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line by tabs
    elements = line.split("\t")
    # The 1st value in the line is the ID of the document
    label = elements[0]
    # Update the 'count_docs' variable with the numbers from the file for each class
    if label == 'count,docs':
        count_docs[0] += int(elements[1])
        count_docs[1] += int(elements[2])
    # Update the 'count_words' variable with the numbers from the file for each class
    elif label == 'count,words':
        count_words[0] += int(elements[1])
        count_words[1] += int(elements[2])
    # Update the 'count_vocab_terms' variable with the numbers from the file for each class
    else:
        term = label
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [int(elements[1]), int(elements[2])]
        else:
            count_vocab_terms[term][0] += int(elements[1])
            count_vocab_terms[term][1] += int(elements[2])

# Per HW2.5 instructions, update vocabulary to ignore terms that have frequency less than 3.
# Create a flag to update the total word counts after removing the infrequent terms.
UPDATE_WORD_COUNTS = True
for term in count_vocab_terms.keys():
    # store the counts of the term for each class
    count_0 = count_vocab_terms[term][0]
    count_1 = count_vocab_terms[term][1]
    # If a term has a frequency of less than 3, then remove it from the vocabulary.
    if (count_0 + count_1) < 3:
        if UPDATE_WORD_COUNTS:
            count_words[0] -= count_0
            count_words[1] -= count_1
        # Remove the term
        del count_vocab_terms[term]
            
# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Use smoothing by adding 1.0 to the numerator and the denominator in the condprob equation.
# Create a variable to control if smoothing is used.
# For 2.4 and 2.5 set SMOOTHING to True
SMOOTHING = True
# Print the prior probabilities for each class
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
print('prior,prob\t%f\t%f' % (prior[0], prior[1]))

# Print the conditional probabilities for each term for Ham and Spam
for term in count_vocab_terms:
    for cls in classes:
        # If using smoothing, then add 1.0 to the numerator and the denominator in the condprob equation.
        if SMOOTHING:
            condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)
        # If NOT using smoothing, then leave the numerator and the denominator alone.
        else:
            condprob[term][cls] = (count_vocab_terms[term][cls]) / (float(count_words[cls]))
    print('%s\t%f\t%f' % (term, condprob[term][0], condprob[term][1]))

Overwriting reducer.py


In [14]:
# Remove output from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-5a/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-5a
# Run a hadoop streaming job to create the model
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-5a

16/05/24 21:43:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 21:43:27 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-5a/_SUCCESS
16/05/24 21:43:27 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-5a/part-00000
16/05/24 21:43:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 21:43:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar1628065760896304683/] [] /tmp/streamjob1728063881307997834.jar tmpDir=null
16/05/24 21:43:30 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.2

In [15]:
# Reivew the output files obtained from the above job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-5a
# Look at the contents of the output of the reducer in the part-00000 file in HDFS
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-5a/part-00000

16/05/24 21:44:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 21:43 /user/hadoop/outputHW2-5a/_SUCCESS
-rw-r--r--   3 hadoop supergroup      46549 2016-05-24 21:43 /user/hadoop/outputHW2-5a/part-00000
16/05/24 21:44:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
prior,prob	0.560000	0.440000
increase	0.000560	0.000450
regional	0.000960	0.000064
four	0.000320	0.000386
wednesday	0.000560	0.000064
tires	0.000080	0.000257
second	0.000160	0.000257
errors	0.000160	0.000386
bonuses	0.000080	0.000322
increasing	0.000400	0.000064
affiliates	0.000080	0.000450
k	0.000160	0.000257
i'm	0.000320	0.000322
unit	0.000640	0.000064
derivatives	0.000320	0.000064
therefore	0.000320	0.000322
until	0.001040	0.000386
successful	0.000080	0.001158
brings	0.000240	0.000257
yah

In [16]:
# Copy the model output from the 1st reducer to the local file system
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-5a/part-00000 model25.txt

16/05/24 21:44:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 21:44:42 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [17]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 2 code for HW2.5

# Import print function from python 3
from __future__ import print_function
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Initialize the 'condprob' variable to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}

# Load the model from the model.txt file.
# 1.  prior:  stores the prior probabilites for Ham (0) and Spam (0)
# 2.  condprob:  stores the conditional probabilities for each term for Ham (0) and Spam (0)
with open("model25.txt", 'r') as f:
    for line in f.readlines():
        elements = line.split('\t')
        if elements[0] == 'prior,prob':
            prior = [float(elements[1]), float(elements[2])]
        else:
            term = elements[0]
            condprob[term] = [float(elements[1]), float(elements[2])]

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]

for line in sys.stdin:
    # Read in the lines from the input file
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    # This bypasses line 60 which is only a partial input line, so it is ignored
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Make a prediction for each document based on the model
    score = [0, 0]
    for cls in classes:
        # Initialize the score with the prior probability for the class
        score[cls] = math.log(prior[cls])
        for term in words_all:
            # If the term is in the vocabulary, then add the conditional probability to the score.
            if term in condprob:
                # Add the conditional probability to the scoare for each term.
                # If any of the conditional probabilities for the document are 0, then set the score
                # to -100000000 because, in actuality the probabilities are being multiplied, so if one is 0
                # then the whole score will be 0.  If this happens, set the score to a very low number so that
                # this class will have a very low score and won't be selected and break out of the loop
                if condprob[term][cls] == 0.0:
                    score[cls] = -1000000000
                    count_docs_zero_prob[cls] += 1
                else:
                    score[cls] += math.log(condprob[term][cls])
    # Set the prediction for the document to the class that has the highest score
    prediction = 1 if (score[1] > score[0]) else 0
    # Print the document ID, the real classification, the predicted classification and the score
    # for each class for the reducer
    print("%s\t%d\t%d\t%f\t%f" % (ID, TRUTH, prediction, score[0], score[1]))

# Print the number of times a zero probability for reporting purposes.
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))

Overwriting mapper.py


In [18]:
%%writefile reducer.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Reducer 2 code for HW2.5

import sys

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]
# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Initialize a dictionary to store the IDs and the scores from the predictions
doc_probs = {}

# Input comes from STDIN
for line in sys.stdin:
    # Split the line by tabs
    items = line.split('\t')
    # Store the zero probability counts
    if items[0] == 'count,zeroprob':
        count_docs_zero_prob[0] += int(items[1])
        count_docs_zero_prob[1] += int(items[2])
    # Update the incorrect and total counts based on the accuracy of the prediction
    # Store the scores for each class in the doc_probs dictionary for reporting purposes
    else:
        ID = items[0]
        TRUTH = int(items[1])
        prediction = int(items[2])
        if TRUTH != prediction:
            count_incorrect += 1
        count_total += 1
        doc_probs[ID] = [float(items[3]), float(items[4])]

# Print the Training Error, zero probability counts and scores for reporting purposes
print('Training Error: %f' % (count_incorrect / float(count_total)))
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))
for ID, probs in doc_probs.iteritems():
    print('%s\t%f\t%f' % (ID, probs[0], probs[1]))

Overwriting reducer.py


In [19]:
# Remove the outputs from previous mapreduce jobs
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-5b/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-5b
# Call a hadoop streaming job to classify the documents.  Pass in the model.txt file so it is available
# to all of the mappers
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py,model25.txt -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-5b

16/05/24 21:45:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 21:45:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-5b/_SUCCESS
16/05/24 21:45:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-5b/part-00000
16/05/24 21:45:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/24 21:45:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar3049769881873987326/] [] /tmp/streamjob598717389619044906.jar tmpDir=null
16/05/24 21:45:32 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.20

In [20]:
# Inspect the output produced from the mapreduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-5b
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-5b/part-00000

16/05/24 21:45:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-24 21:45 /user/hadoop/outputHW2-5b/_SUCCESS
-rw-r--r--   3 hadoop supergroup       4796 2016-05-24 21:45 /user/hadoop/outputHW2-5b/part-00000
16/05/24 21:45:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Training Error: 0.020000	
count,zeroprob	0	0
0010.2003-12-18.GP	-15.169600	-16.658759
0010.2001-06-28.SA_and_HP	-2849.832563	-2580.608804
0001.2000-01-17.beck	-2769.597025	-3286.915589
0018.1999-12-14.kaminski	-806.859479	-812.082096
0005.1999-12-12.kaminski	-439.889532	-521.289855
0011.2001-06-29.SA_and_HP	-14970.017660	-13156.279116
0008.2004-08-01.BG	-3703.990612	-3474.178015
0009.1999-12-14.farmer	-383.937997	-450.869066
0017.2003-12-18.GP	-159.088673	-147.322627
0011.2001-06-28.SA_and_H

In [21]:
# Copy the model output from the 2nd reducer to the local file system for reporting purposes.
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-5b/part-00000 doc_scores25.txt

16/05/24 21:46:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
copyToLocal: `doc_scores25_no_update.txt': File exists


**HW2.6** Benchmark your code with the Python SciKit-Learn implementation of the multinomial Naive Bayes algorithm.  

In this exercise, please complete the following:

**2.6.1.** Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW2.5 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)

ANSWER:  MultinomialNB - Scikit Learn with Smoothing Training Error: 0.020000

**2.6.2.** Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop implementation) and the column presents the training misclassification error.

ANSWER:

| Algorithm | Training Error |
| ------ | ----------- |
| MultinomialNB - SciKit-Learn   | 2% |
| MultinomialNB - Hadoop Streaming | 2% |

**2.6.3.** Explain/justify any differences in terms of training error rates over the dataset in HW2.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn 

ANSWER:  There is no difference between the training error rate with the HW2.5 solution and the SciKit-Learn Multinomial Naive Bayes implemention.  The training error rates are the same.  This makes sense for the following reasons:  1.  Add-1 smoothing was used in both cases.  2.  The words fed in to each algorithm were the same.  The text was first parsed in the same way and a regular expression was used to break down the text in to words.  The reduced vocabulary where only words that appeared at least 3 times was used for both algorithms.  (To enable this for the SciKit-Learn algorithm, the vocabulary from the model in HW2.5 was read in and passed to the CountVectorizer function so that only the tokens from the reduced vocabulary were used to train and test the model.)

In [1]:
# Import SK-learn libraries for learning.
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
# Import SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Load the model from the model.txt file.  Extrat the vocabulary from HW2.5 into the vocab list
vocab = []
with open("model25.txt", 'r') as f:
    for line in f.readlines():
        elements = line.split('\t')
        if elements[0] == 'prior,prob':
            pass
        else:
            vocab.append(elements[0])
            
# Transform the training data into a format that the sklearn libraries can use.
# train_ids:  stores the IDs from the training data.
train_ids = []
# train_data:  stores the text from each document
train_data = []
# train_labels:  stores the true labels for each document
train_labels = []
# set filename to the file that is being analyzed
filename = "enronemail_1h.txt"
# Open the file
with open (filename, "r") as myfile:
    # Loop through each line in the file
    for line in myfile.readlines():
        # Split the lines by tab
        items = line.split('\t')
        # This bypasses line 60 which is only a partial input line, so it is ignored
        if len(items) == 1:
            continue
        # Add each ID to the 'train_ids' list
        train_ids.append(items[0])
        # Add each label to the 'train_labels' list
        train_labels.append(int(items[1]))
        # The 3rd value of the line is the subject.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        words_all = re.findall(WORD_RE, items[2].lower())
        # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
        # break it up into separate words using the WORD_RE regular expression
        if len(items) == 4:
            words_all = words_all + re.findall(WORD_RE, items[3].lower())
        # Create one string containing all of the words in the document separated by spaces
        # and then add that string to the 'train_data' list
        train_data.append(' '.join(words_all))

# Create a CountVectorizer object that will transform the text data into a feature vector
cv = CountVectorizer(vocabulary=vocab)
# Transform the train_data into a feature vector
train_fv = cv.fit_transform(train_data)

##### PART 2.6.1 #######
# Create a MultinomialNB model using the default parameters
multiNB = MultinomialNB()
# Fit the model with the training data
multiNB.fit(train_fv, train_labels)
# Create predictions from the model using the training data
predict = multiNB.predict(train_fv)

# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
# Calculate the training error by dividing the count_incorrect by the total count
for i in range(len(predict)):
    if predict[i] != train_labels[i]:
        count_incorrect += 1
    count_total += 1
print('PART 2.6.1')
print('MultinomialNB - SciKit-Learn with smoothing Training Error: %f' % (count_incorrect / float(count_total)))
print('')

##### PART 2.6.2 #######
# Import tabulate to create a table
from tabulate import tabulate
# Create a variable 'headers' to store the headers of the table
headers = ['   ', 'Training Error']
# Initialize a list called data to hold the data for the rows of the table
data = [['MultinomialNB - SciKit-Learn', '2%'],
        ['MultinomialNB - Hadoop Streaming', '2%']]

# Print the table
print('PART 2.6.2')
print(tabulate(data, headers=headers))
print('')

PART 2.6.1
MultinomialNB - SciKit-Learn with smoothing Training Error: 0.020000

PART 2.6.2
                                  Training Error
--------------------------------  ----------------
MultinomialNB - SciKit-Learn      2%
MultinomialNB - Hadoop Streaming  2%



In [4]:
import re
WORD_RE = re.compile(r"[\w']+")
items = "i am here, now. and then! http://test.com"
words_all = re.findall(WORD_RE, items.lower())
words_all1 = re.split('\s|\,|\.',(items.lower()))
words_all2 = re.split('[\s|\.|\,]+',(items.lower()))
print(words_all)
print(words_all1)
print(words_all2)

['i', 'am', 'here', 'now', 'and', 'then', 'http', 'test', 'com']
['i', 'am', 'here', '', 'now', '', 'and', 'then!', 'http://test', 'com']
['i', 'am', 'here', 'now', 'and', 'then!', 'http://test', 'com']
